## Dieses Skript wertet die Daten inklusive der Daten aus den Wochen, in welchen fehlerhafte Daten vorliegen, aus

In [ ]:
### Importieren der verwendeten Bibilotheken
import os
import glob
from collections import Counter

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
#Plot Style muss zur Verwendung hinterlegt werden
#plt.style.use(['mystyle'])

### Einlesen des zuvor erstellten Datensatzes

In [ ]:
pathToPickle = os.path.join("..", "..", "data", "TUDA_battery_state", "all_TUDA_battery_states_battery_state.pickle")
df = pd.read_pickle(pathToPickle)

## Aufzeigen wieviele Batterieladezustände 0% sind
Vermutlich zurückzuführen auf korrupted Data

In [ ]:
arrayBatterieNullUngleichNull = []
bins_seconds_labels = df.Tag.unique()
for label in bins_seconds_labels:
    # Anzahl der Werte einer Woche (labels) werden zurückgegeben, deren 'battery_state' gleich 0 ist 
    zerosNumbers = df['battery_state'].loc[df['Woche']==label].loc[df['battery_state'].loc[df['Woche']==label] == 0].size
    # Anzahl der Werte einer Woche (labels) werden zurückgegeben, deren 'battery_state' ungleich 0 ist 
    notZerosNumbers = df['battery_state'].loc[df['Woche']==label].loc[df['battery_state'].loc[df['Woche']==label] != 0].size
    # an das die Liste 'arrayBatterieNullUngleichNull' wird eine Liste mit der Woche (label) und den berechneten Anzahlen angehängt
    arrayBatterieNullUngleichNull.append([label,zerosNumbers,notZerosNumbers])

# DataFrame mit einzelnen Wochen und den Anzahlen der 0 Werte der Batterielaufzeiten und den !=0 Werten der Batterielaufzeiten
dfBatterieNullUngleichNull = pd.DataFrame(arrayBatterieNullUngleichNull, columns=['week', 'equalsZero', 'notZero'])

# Zeilen, löschen, an denen Beide Were 0 sind
indexZero = dfBatterieNullUngleichNull[(dfBatterieNullUngleichNull['equalsZero']==0) & (dfBatterieNullUngleichNull['notZero']==0)].index
indexZero
dfBatterieNullUngleichNull.drop(indexZero, inplace=True)

### Für alle Wagontypen und Provider

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize = (7,5))
dfBatterieNullUngleichNull.plot.bar(x = 'week', y = ['equalsZero','notZero'], ax = ax, width=1, stacked=True)
ax.set_xlabel('Woche')

labels = []
for i in range(dfBatterieNullUngleichNull['week'].size):
    if (i%4 == 0):
        labels.append(i+1)
    else:
        labels.append('')

ax.set_xticklabels(labels=labels)        
ax.set_ylabel('Anzahl an Datenpunkten')
ax.legend(labels=['Batterieladezustand $= 0 \%$', r'Batterieladezustand $\neq 0 \%$'])
ax.grid(False)

### Aufteilen des DataFrames in Wagontypen

In [ ]:
# Aufteilen des DataFrames in Wagontypen
pathToMappingPickle = os.path.join("..", "..", "data", "mappingDf.pickle")
mappingdf = pd.read_pickle(pathToMappingPickle)
df = pd.merge(df, mappingdf, on = "wagon_ID")


In [ ]:
wagon_typeList = np.sort(df["wagon_type"].drop_duplicates().to_numpy())
wagon_typeList

In [ ]:
dfs_wagon_type = []
for i, wagon_type in enumerate(wagon_typeList):
    df_type = df[df['wagon_type'] == wagon_type]
    dfs_wagon_type.append(df_type)
    
    
    

In [ ]:
def Batteriezustandswerte(tempdf):
    # bins werden erstellt, für jeden Woche zwischen 0 und 322, in Sekunden, da die Daten in Sekunden vorliegen
    bins_seconds = np.arange(0, 47*7*24*60*60, 7*24*60*60)
    # liste mit bin labels für jede Woche erstellen
    bins_seconds_labels = [i+1 for i, bin in enumerate(bins_seconds)]
    # bins werden aus numpy array in Liste umgewandelt
    bins_seconds = bins_seconds.tolist()
    # letzter Wert für die labels wird gelöscht, da ein Label mehr als bins 
    bins_seconds_labels.pop()

    # Daten mittels der timestamps in die erstellten bins eingeteilt (jeweils immer 1 Woche einen bin zuweisen) und entsprechendes Label wird
    # dem DataFrame angehängt
    tempdf['bin'] = pd.cut(tempdf['timestamp_measure_battery'], bins = bins_seconds, labels = bins_seconds_labels)
    

    arrayBatterieNullUngleichNull = []

    for label in bins_seconds_labels:
        # Anzahl der Werte einer Woche (labels) werden zurückgegeben, deren 'battery_state' gleich 0 ist 
        zerosNumbers = tempdf['battery_state'].loc[tempdf['bin']==label].loc[tempdf['battery_state'].loc[tempdf['bin']==label] == 0].size
        # Anzahl der Werte einer Woche (labels) werden zurückgegeben, deren 'battery_state' ungleich 0 ist 
        notZerosNumbers = tempdf['battery_state'].loc[tempdf['bin']==label].loc[tempdf['battery_state'].loc[tempdf['bin']==label] != 0].size
        # an das die Liste 'arrayBatterieNullUngleichNull' wird eine Liste mit der Woche (label) und den berechneten Anzahlen angehängt
        arrayBatterieNullUngleichNull.append([label,zerosNumbers,notZerosNumbers])

    # DataFrame mit einzelnen Wochen und den Anzahlen der 0 Werte der Batterielaufzeiten und den !=0 Werten der Batterielaufzeiten
    dfBatterieNullUngleichNull = pd.DataFrame(arrayBatterieNullUngleichNull, columns=['week', 'equalsZero', 'notZero'])
    return dfBatterieNullUngleichNull



In [ ]:
dfs_wagon_types_batteriezustand = []
for df_wagon_type in dfs_wagon_type:
    dfs_wagon_types_batteriezustand.append(Batteriezustandswerte(df_wagon_type))


### Plotten der Anzahl an Datenpunkten bezogen auf Wagon Type

In [ ]:
fig, ax = plt.subplots(ncols = 4, nrows = 2, figsize = (20,10), sharey=True)

ax = ax.flatten()
for i, df_wagon_types_batteriezustand in enumerate(dfs_wagon_types_batteriezustand):
    df_wagon_types_batteriezustand.plot.bar(x = 'week', y = ['equalsZero','notZero'], ax=ax[i],width=1, stacked=True)
    ax[i].set_xlabel('Woche')

    labels = []
    for k in range(df_wagon_types_batteriezustand['week'].size):
        if (k%4 == 0):
            labels.append(k+1)
        else:
            labels.append('')
            
    ax[i].set_xticklabels(labels=labels)        
    ax[i].set_ylabel('Anzahl an Datenpunkten')
    ax[i].grid(False)
    ax[i].set_title(f'Wagon Typ: {i+1}')
    if i == 0:
        ax[0].legend(labels=['Batterieladezustand $= 0 \%$', r'Batterieladezustand $\neq 0 \%$'])
    else:
        ax[i].legend(labels=[])

### Daten nach Providern aufteilen, Hinweis: der Provider 0 wird nicht beachtet, es handelt sich um keinen Provider (vermutliche fehlerhafte Werte)

In [ ]:
providerList = np.sort(df["provider_y"].drop_duplicates().to_numpy())
providerList = np.delete(providerList ,0)

In [ ]:
providerList

In [ ]:
dfs_provider = []
for i, provider in enumerate(providerList):
    df_type = df[df['provider_y'] == provider]
    dfs_provider.append(df_type)
    

In [ ]:
dfs_provider_batteriezustand = []
for df_provider in dfs_provider:
    dfs_provider_batteriezustand.append(Batteriezustandswerte(df_provider))


### Plotten der Anzahl an Datenpunkten bezogen auf Provider

In [ ]:
fig, ax = plt.subplots(ncols = 3, nrows = 1, figsize = (7,3), constrained_layout = True, sharey = True)
ax = ax.flatten()

for i, df_provider_batteriezustand in enumerate(dfs_provider_batteriezustand):
    df_provider_batteriezustand.plot.bar(x = 'week', y = ['equalsZero','notZero'], ax=ax[i],width=1, stacked=True)
    ax[i].set_xlabel('Woche')

    labels = []
    k = 0
    for k in range(df_provider_batteriezustand['week'].size):
        if (k%4 == 0):
            labels.append(k+1)
        else:
            labels.append('')
            
    ax[i].set_xticklabels(labels=labels)        
    ax[i].set_ylabel('Anzahl an Datenpunkten')
    ax[i].grid(False)
    ax[i].set_title(f'Provider {providerList[i]}')
        
        
    #if i == 1:
    #    ax[1].legend(labels=['Batterieladezustand $= 0 \%$', r'Batterieladezustand $\neq 0 \%$'])
    #else:
   #     ax[i].get_legend().remove()
    ax[i].get_legend().remove()
legend_box = (0.05, 1.03, 0.95, 0.7)
legend = fig.legend(fontsize='small', bbox_to_anchor=legend_box, ncol=2, loc=3, borderaxespad=0.0, mode='expand', handlelength=1,labels=['Batterieladezustand $= 0 \%$', r'Batterieladezustand $\neq 0 \%$'])

# Wenn Abbildung gespeichert werden soll, Pfad mit anegeben!
#fig.savefig(os.path.join('..', '..', 'data', 'Batteriezustaende.png'))

### Analyse der Zeitabstände zwischen einzelnen Batteriemessungen

In [ ]:
wagon_IDList = df["wagon_ID"]
wagon_IDList = wagon_IDList.drop_duplicates()
wagon_IDList = wagon_IDList.to_numpy()
wagon_IDList = np.sort(wagon_IDList)

In [ ]:
timeNumpy = []
timeDiff = []
timeList = []


for i,idx in enumerate(wagon_IDList):
    wagondf = df.loc[df['wagon_ID'] == idx]
    wagondf = wagondf.sort_values("timestamp_measure_battery")
    timeNumpy =  wagondf.timestamp_measure_battery.to_numpy()
    # wagondf.sub('timestamp_measure_battery')
    timeDiffSingle = np.diff(timeNumpy)
    timeDiff.append(timeDiffSingle)

timeDiff = [np.floor(x / (24*60*60)) for x in timeDiff]

[timeList.extend(e) for e in timeDiff[:]]
countTime = Counter(timeList)


In [ ]:
len(timeList)

In [ ]:
countTimeList = countTime.most_common()
ii=-1
firstRow =  [0] * len(countTimeList)
secRow   =  [0] * len(countTimeList)

for i in countTimeList:
    ii += 1
    firstRow[ii] = countTimeList[ii][0]
    secRow[ii]   = countTimeList[ii][1]

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 3))

# ax.set_title(('Abstände der gesendeten Batteriezustände in Tagen'))
ax.set_ylabel('Anzahl der Sendungen')
ax.set_xlabel('Zeitraum in Tagen')

ax.grid(False)

ax.bar(firstRow[0:15],secRow[0:15])
# Wenn plot gespeichert werden soll, sinnvollen Pfad angeben
#fig.savefig(os.path.join('..', '..', 'data', 'Abstaende_Sendungen.png'))